In [3]:
import os
import numpy as np
import re
import pickle
import sys
import pandas as pd

pd.set_option('display.float_format', lambda x: '%.6f' % x)

import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

sys.path.insert(1, '../../../ProcessEvents')
from ProcessEventsFunctions import *
# from Convert_to_Profiles_Functions import *
# from Get_Events_Functions import *

In [4]:
home_dir = '/nfs/a319/gy17m2a/PhD/'
home_dir2 = '/nfs/a161/gy17m2a/PhD/'

### Check number of years covered by each time period

In [5]:
print("UKCP18 Present", len(list(range(2001,2020))))
print("UKCP18 Future", len(list(range(2061,2080))))
print("NIMROD: ", len(list(range(2006,2021))))

UKCP18 Present 19
UKCP18 Future 19
NIMROD:  15


### Check if there is the same number of files in the folder containing all events

For each gauge, count the number of files (part0 in name, so not including the CAM)

Answer = 19 * 1291 * 7 = 171703

In [ ]:
# durations =[0.5,1,2, 3,6,12, 24]
# for duration in durations:
#     part0s = [item for item in files if 'part1' not in item]
#     print(len([item for item in part0s if item.startswith(f'{duration}hr')]))

In [41]:
time_period='Present'
ems_present = ['bc005', 'bc006', 'bc007', 'bc009', 'bc010', 'bc011', 'bc012', 'bc013', 'bc015', 'bc016', 'bc017', 'bc018']
cams_dict_present ={}
events_dict_present={}
for em in ems_present:
    cams_thisem = 0
    events_thisem = 0    
    for gauge_num in range(0, 1294):
        if gauge_num not in [444, 827, 888]:
            indy_events_fp = home_dir2 + f"ProcessedData/IndependentEvents/UKCP18_30mins/{time_period}/{em}/{gauge_num}/WholeYear/EventSet"
            files = [f for f in os.listdir(indy_events_fp) if f.endswith('.csv')]
            files = np.sort(files)
            if len([item for item in files if 'part1' not in item])!= 133:
                print(em, gauge_num, len([item for item in files if 'part1' not in item]))
            for item in files:
                if 'part1' in item:
                    cams_thisem=cams_thisem+1
                elif 'part0' in item:
                    events_thisem = events_thisem+1                          
            cams_dict_present[em] = cams_thisem
            events_dict_present[em] = events_thisem         
events_dict_present            

{'bc005': 171703,
 'bc006': 171703,
 'bc007': 171703,
 'bc009': 171703,
 'bc010': 171703,
 'bc011': 171703,
 'bc012': 171703,
 'bc013': 171703,
 'bc015': 171703,
 'bc016': 171703,
 'bc017': 171703,
 'bc018': 171703}

In [7]:
time_period='Future'
ems_future = ['bb195', 'bb192', 'bb198', 'bb208', 'bb225','bb222', 'bb201', 'bb204', 'bb216', 'bb219', 'bb211']

cams_dict_future={}
events_dict_future={}
for em in ems_future:
    cams_thisem = 0
    events_thisem = 0
    for gauge_num in range(0, 1294):
        if gauge_num not in [444, 827, 888]:
            indy_events_fp = home_dir2 + f"ProcessedData/IndependentEvents/UKCP18_30mins/{time_period}/{em}/{gauge_num}/WholeYear/EventSet/"
            files = [f for f in os.listdir(indy_events_fp) if f.endswith('.csv')]
            files = np.sort(files)
            if len([item for item in files if 'part1' not in item])!= 133:
                print(em, gauge_num, len([item for item in files if 'part1' not in item]))
            for item in files:
                if 'part1' in item:
                    cams_thisem=cams_thisem+1
                elif 'part0' in item:
                    events_thisem=events_thisem+1                    
            cams_dict_future[em] = cams_thisem
            events_dict_future[em] = events_thisem       
events_dict_future            

bb198 313 132


{'bb195': 171703,
 'bb192': 171703,
 'bb198': 171702,
 'bb208': 171703,
 'bb225': 171703,
 'bb222': 171703,
 'bb201': 171703,
 'bb204': 171703,
 'bb216': 171703,
 'bb219': 171703,
 'bb211': 171703}

### NIMROD
Expecting 15 * 1291 *7 = 135555

In [48]:
cams_thisem = 0
events_thisem = 0    
for gauge_num in range(0, 1294):
    if gauge_num not in [444, 827, 888]:
        indy_events_fp = home_dir2 + f"ProcessedData/IndependentEvents/NIMROD_30mins/NIMROD_2.2km_filtered_100/{gauge_num}/WholeYear/EventSet"
        files = [f for f in os.listdir(indy_events_fp) if f.endswith('.csv')]
        files = np.sort(files)
        if len([item for item in files if 'part1' not in item])!= 105:
            print(gauge_num, len([item for item in files if 'part1' not in item]))
        for item in files:
            if 'part1' in item:
                cams_thisem=cams_thisem+1
            elif 'part0' in item:
                events_thisem=events_thisem+1        
events_thisem                

135555

### Check if there are enough files for every duration
This is not really necessary, because if it wasn't 19 then numbers above would be wrong as well.  
Counting just events with 'part0' otherwise will be different due to CAM - should always be 19

In [44]:
durations = ['0.5', '1', '2', '3', '6', '12', '24']

In [45]:
time_period = 'Present'
for em in ['bc017']:
    print(em)
    for gauge_num in range(0, 1294):
        if gauge_num not in [444, 827, 888]:
            # print(f"gauge num {gauge_num}")
            indy_events_fp = home_dir2 + f"ProcessedData/IndependentEvents/UKCP18_30mins/{time_period}/{em}/{gauge_num}/WholeYear/EventSet/"
            files = [f for f in os.listdir(indy_events_fp) if f.endswith('.csv')]
            files = np.sort(files)
            for duration in durations:
                count_starts = sum(s.startswith(f"{duration}hrs") and 'part0' in s for s in files)
                # print(duration, count_starts)
                if count_starts != 19:
                    print('broken', count_starts, duration)
                    break

bc017
